In [ ]:
# api 설치
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:

from fastapi import FastAPI
from pydantic import BaseModel
from contextlib import asynccontextmanager
from module.model import load_models, predict

c:\Users\kase\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# lifespan으로 모델 로드 관리
@asynccontextmanager
async def lifespan(app: FastAPI):
    global models, tokenizers, device
    models, tokenizers, device = load_models()
    yield
    models.clear()
    tokenizers.clear()

In [13]:
app = FastAPI(title="KLUEBERT Multi-Model API", lifespan=lifespan)

In [14]:
class PredictInput(BaseModel):
    text: str

# 분석조회 테이블 출력
# 현재 감지모델 출력만 있음

@app.post("/api/v1/channels/{channelId}/reports/frequency")
def get_prediction(data: PredictInput):
    results = {}
    for disease in ["depression", "anxiety"]:
        results[disease] = predict(data.text, models[disease], tokenizers[disease], device)
    return {"input": data.text, "predictions": results}

In [15]:
# 직동확인 부분
@app.get("/")
def root():
    return {"message": "KLUEBERT API is running", "models": ["depression", "anxiety"]}